# Zadanie 1
Pobieraj dane z pliku https://github.com/przem85/statistics/blob/master/D8/ANOVA4.txt
Zawiera on dane z eksperymentu na roślinach, które były hodowane w trzech  różnych warunki wzrostu. 

- Wykonaj ANOVA
- Czy trzy grupy są różne?
- Wykonaj analizę post hoc, który z par jest inny? 
- Czy porównanie nieparametryczne (Kruskal-Wallis test) prowadzi do innego wyniku? 

In [13]:
import numpy as np
import scipy.stats as stats
import pandas as pd

In [14]:
df = pd.read_csv('ANOVA4.txt',sep=" ",header=None, names=['group','weight'])
df.head()
#df['group'].dtype

group  weight
0  Control    4.17
1  Control    5.58
2  Control    5.18
3  Control    6.11
4  Control    4.50

In [15]:
 # Sort them into groups, according to column 1
data=np.array(df)
group1 = data[data[:,0]=='Control',1]
group2 = data[data[:,0]=='TreatmentA',1]
group3 = data[data[:,0]=='TreatmentB',1]

In [16]:
# g_a = df['weight'][df['group']=='TreatmentA']
# g_b = df['weight'][df['group']=='TreatmentB']
# g_c = df['weight'][df['group']=='Control']

## Saprawdzamy założenie o równości variancii 

In [17]:
# First, check if the variances are equal, with the "Levene"-test
(W,p) = stats.levene(group1, group2, group3)
if p<0.05:
    print(('Warning: the p-value of the Levene test is <0.05: p={0}'.format(p)))
else:    
    print(('OK: the p-value of the Levene test is >0.05: p={0}'.format(p)))

OK: the p-value of the Levene test is >0.05: p=0.32927821561008164


## Wykonuje ANOWE jednoczynnikową 

In [18]:
# Do the one-way ANOVA
F_statistic, pVal = stats.f_oneway(group1, group2, group3)

In [19]:
print((F_statistic, pVal))
if pVal < 0.05:
    print('One of the groups is significantly different.')

(6.005680376516912, 0.00695554199802733)
One of the groups is significantly different.


## Bardziej szczegółowym wynikiem ANOVA można otrzymać za pomom modelowania statystycznego:

In [20]:
data1=data
data1[data1[:,0]=='Control',0]=1.
data1[data1[:,0]=='TreatmentA',0]=2.
data1[data1[:,0]=='TreatmentB',0]=3.
data1[:,0].astype(float)
data1[:,1].astype(float)
df1=pd.DataFrame(data1, columns=['group','weight'])
df1.head(100)

group weight
0      1   4.17
1      1   5.58
2      1   5.18
3      1   6.11
4      1    4.5
5      1   4.61
6      1   5.17
7      1   4.53
8      1   5.33
9      1   5.14
10     2   4.81
11     2   4.17
12     2   4.41
13     2   3.59
14     2   5.87
15     2   3.83
16     2   6.03
17     2   4.89
18     2    4.3
19     2   4.69
20     3   6.31
21     3    5.1
22     3   5.54
23     3    5.5
24     3   5.37
25     3    5.9
26     3    4.9
27     3   6.15
28     3    5.8
29     3    5.6

In [21]:
# First, check if the variances are equal, with the "Levene"-test
(W,p) = stats.levene(group1, group2, group3)
if p<0.05:
    print(('Warning: the p-value of the Levene test is <0.05: p={0}'.format(p)))
else:
    print(('OK: the p-value of the Levene test is >0.05: p={0}'.format(p)))

OK: the p-value of the Levene test is >0.05: p=0.32927821561008164


In [22]:
print(np.mean(group1)) 
print(np.mean(group2))
print(np.mean(group3))

5.032
4.659
5.616999999999999


In [23]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
# Elegant alternative implementation, with pandas & statsmodels
model = ols('weight ~ C(group)', df).fit()
anovaResults = anova_lm(model)
print(anovaResults)


            df     sum_sq   mean_sq        F    PR(>F)
C(group)   2.0   4.663727  2.331863  6.00568  0.006956
Residual  27.0  10.483460  0.388276      NaN       NaN


In [24]:
from statsmodels.stats import multicomp
mc = multicomp.MultiComparison(df['weight'], df['group'])
print(mc.tukeyhsd().summary())

# Show the group names


    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1     group2   meandiff p-adj   lower  upper  reject
-----------------------------------------------------------
   Control TreatmentA   -0.373  0.388 -1.0638 0.3178  False
   Control TreatmentB    0.585 0.1089 -0.1058 1.2758  False
TreatmentA TreatmentB    0.958 0.0053  0.2672 1.6488   True
-----------------------------------------------------------


In [19]:
print('\n Kruskal-Wallis test ----------------------------------------------------')


# Then do the Kruskal-Wallis test
h, p = stats.kruskal(g_c, g_a, g_b)
print('Result from Kruskal-Wallis test: p = {0}'.format(p))


 Kruskal-Wallis test ----------------------------------------------------
Result from Kruskal-Wallis test: p = 0.011030052001273464


In [21]:
df['group']

0        Control
1        Control
2        Control
3        Control
4        Control
5        Control
6        Control
7        Control
8        Control
9        Control
10    TreatmentA
11    TreatmentA
12    TreatmentA
13    TreatmentA
14    TreatmentA
15    TreatmentA
16    TreatmentA
17    TreatmentA
18    TreatmentA
19    TreatmentA
20    TreatmentB
21    TreatmentB
22    TreatmentB
23    TreatmentB
24    TreatmentB
25    TreatmentB
26    TreatmentB
27    TreatmentB
28    TreatmentB
29    TreatmentB
Name: group, dtype: object